# Create final datasets with selected Features: PV, Gross load, Total load, Prosumption

This notebook creates the final datasets with the selected features: 
S['hour sin', 'hour cos', 'temp', 'rhum', 'wspd', 'PC1', 'lag_24hrs']

In [1]:
#Imports

#Data handling
import pandas as pd
#Create data arrays
import numpy as np
#Machine learning models
import tensorflow as tf
#Plotting
import matplotlib.pyplot as plt
#Create Folder for modelling checkpoint
import os
#Normalization
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

import sys  
sys.path.append("../../../")  
from src.utils.modelgenerator import *

In [2]:
cwd = os.path.normpath(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

In [3]:
def check_data(df):
    # Check for negative values
    negative_counts = (df.iloc[:, :] < 0).any()

    # Check for NaN values
    nan_counts = (df.iloc[:, :].isna()).any()

    # Check if there are any users with negative values or NaN values
    if negative_counts.any() or nan_counts.any():
        print("Users with negative values or NaN:")
        print("Negative counts:")
        print(negative_counts[negative_counts].index)
        print("NaN counts:")
        print(nan_counts[nan_counts].index)
    else:
        print("No negative values and no NaN values.")

In [4]:
#Principal Componant Analysis

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pca_analysis(df, n_components=1):
    
    # Extract the weather features from your DataFrame
    weather_features = df[['temp', 'rhum', 'dwpt', 'wdir', 'wspd', 'pres']]

    # Standardize the data
    scaler = StandardScaler()
    scaled_weather_features = scaler.fit_transform(weather_features)

    # Create a PCA instance with the desired number of components
    pca = PCA(n_components=n_components)
    # Fit and transform the scaled data using PCA
    principal_components = pca.fit_transform(scaled_weather_features)

    # Create a DataFrame to store the principal components
    pc_df = pd.DataFrame(data=principal_components, index=df.index, columns=[f'PC{i+1}' for i in range(n_components)])

    # Add the principal components back to your original DataFrame if needed
    df_with_pcs = pd.concat([df, pc_df], axis=1, )
    return df_with_pcs

In [5]:
def sine_cosine_scaling(df):

    #Time Feature Engineering
    df['hour'] = df.index.hour
    #One Sin/ cos cycle takes 2*pi
    df['hour sin'] = np.sin((df['hour']/24)*2 * np.pi)
    df['hour cos'] = np.cos((df['hour']/24)*2 * np.pi)
    df.drop(columns=["hour"], inplace=True)
    return df

In [6]:
def lag_features(df):
    # Specify the User columns
    user_columns = df.filter(like='User').columns

    # Specify the lag intervals in hours
    lag_intervals = [24]

    df_new = df

    # Iterate through each user column and add lagged features
    for user_col in user_columns:
        for lag in lag_intervals:
            # Create a new column with the lagged values
            lag_col_name = f"{user_col}_lag_{lag}hrs"
            df_new[lag_col_name] = df_new[user_col].shift(lag)

    # Drop rows with NaN values created by the lagged features
    df_new.dropna(inplace=True)
    return df_new

# Final PV Dataset

Features ???

In [7]:
pv = pd.read_csv(cwd+'/data/1process_data/2010-2013 Merged_Weather_PV_dataset.csv', index_col='Date')
pv.index = pd.to_datetime(pv.index)
pv

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User297,User298,User299,User300,temp,dwpt,rhum,wdir,wspd,pres
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-01 00:00:00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 01:00:00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 02:00:00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 03:00:00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.7,-1.5,94.0,360.0,7.6,1023.0
2010-07-01 04:00:00,0.0,0.0,0.000,0.0,0.006,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.7,-1.5,94.0,360.0,7.6,1023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-06-30 19:00:00,0.0,0.0,0.006,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.8,9.9,88.0,250.0,3.6,1025.0
2013-06-30 20:00:00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.8,9.9,88.0,250.0,3.6,1025.0
2013-06-30 21:00:00,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.8,8.7,99.0,30.0,1.8,1025.4


In [8]:
check_data(pv)
pv[['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres']] = pv[['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres']].fillna(0)
check_data(pv)

Users with negative values or NaN:
Negative counts:
Index(['temp', 'dwpt'], dtype='object')
NaN counts:
Index(['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres'], dtype='object')
Users with negative values or NaN:
Negative counts:
Index(['temp', 'dwpt'], dtype='object')
NaN counts:
Index([], dtype='object')


In [9]:
#add PCA component
pv = pca_analysis(pv, n_components=1)

#sine cosine scaling
pv = sine_cosine_scaling(pv)

#Drop unsignificant weather features
pv.drop(columns=["dwpt", "wdir", "pres"], inplace=True)

#Introduce Lag Feature (24 hours)
pv = lag_features(pv)

C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[lag_col_name] = df_new[user_col].shift(lag)
C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[lag_col_name] = df_new[user_col].shift(lag)
C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [10]:
display(pv.head(2))
display(pv.tail(2))

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User291_lag_24hrs,User292_lag_24hrs,User293_lag_24hrs,User294_lag_24hrs,User295_lag_24hrs,User296_lag_24hrs,User297_lag_24hrs,User298_lag_24hrs,User299_lag_24hrs,User300_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-02 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-07-02 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User291_lag_24hrs,User292_lag_24hrs,User293_lag_24hrs,User294_lag_24hrs,User295_lag_24hrs,User296_lag_24hrs,User297_lag_24hrs,User298_lag_24hrs,User299_lag_24hrs,User300_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2013-06-30 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-06-30 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Save final dataset
pv.to_csv('../Final_PV_dataset.csv')

# Final Gross Load Dataset

['hour sin', 'hour cos', 'temp', 'rhum', 'wspd', 'PC1', 'lag_24hrs']

In [12]:
gross_load = pd.read_csv(cwd+'/data/1process_data/2010-2013 Merged_Weather_Grossload_dataset.csv', index_col='Date')
gross_load.index = pd.to_datetime(gross_load.index)
gross_load

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User297,User298,User299,User300,temp,dwpt,rhum,wdir,wspd,pres
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-01 00:00:00,0.125,0.179,0.065,0.078,0.000,0.036,0.065,0.146,0.025,0.029,...,0.120,0.129,0.141,0.097,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 01:00:00,0.471,0.142,0.671,0.086,0.000,0.041,0.100,0.088,0.019,0.051,...,0.106,0.110,1.413,0.924,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 02:00:00,0.121,0.119,0.683,0.084,0.000,0.046,0.051,0.097,0.009,0.043,...,0.270,0.105,1.445,0.066,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 03:00:00,0.079,0.088,0.705,0.075,0.000,0.045,0.065,0.104,0.009,0.040,...,0.127,0.063,1.457,0.084,-0.7,-1.5,94.0,360.0,7.6,1023.0
2010-07-01 04:00:00,0.098,0.057,0.817,0.081,0.000,0.045,0.078,0.188,0.015,0.047,...,0.222,0.064,1.418,0.093,-0.7,-1.5,94.0,360.0,7.6,1023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-06-30 19:00:00,0.335,0.465,0.052,0.280,2.152,1.018,0.066,0.563,0.089,0.198,...,0.233,0.366,1.071,1.405,11.8,9.9,88.0,250.0,3.6,1025.0
2013-06-30 20:00:00,0.213,0.317,0.050,0.273,1.993,0.910,0.103,0.104,0.053,0.873,...,0.539,0.318,0.992,0.625,11.8,9.9,88.0,250.0,3.6,1025.0
2013-06-30 21:00:00,0.190,0.224,0.112,0.253,0.814,1.011,0.076,0.086,0.048,0.113,...,0.298,0.272,0.840,0.695,8.8,8.7,99.0,30.0,1.8,1025.4


In [13]:
check_data(gross_load)
gross_load[['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres']] = gross_load[['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres']].fillna(0)
check_data(gross_load)

Users with negative values or NaN:
Negative counts:
Index(['temp', 'dwpt'], dtype='object')
NaN counts:
Index(['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres'], dtype='object')
Users with negative values or NaN:
Negative counts:
Index(['temp', 'dwpt'], dtype='object')
NaN counts:
Index([], dtype='object')


In [14]:
#add PCA component
gross_load = pca_analysis(gross_load, n_components=1)

#sine cosine scaling
gross_load = sine_cosine_scaling(gross_load)

#Drop unsignificant weather features
gross_load.drop(columns=["dwpt", "wdir", "pres"], inplace=True)

#Introduce Lag Feature (24 hours)
gross_load = lag_features(gross_load)

C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[lag_col_name] = df_new[user_col].shift(lag)
C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[lag_col_name] = df_new[user_col].shift(lag)
C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [15]:
display(gross_load.head(2))
display(gross_load.tail(2))

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User291_lag_24hrs,User292_lag_24hrs,User293_lag_24hrs,User294_lag_24hrs,User295_lag_24hrs,User296_lag_24hrs,User297_lag_24hrs,User298_lag_24hrs,User299_lag_24hrs,User300_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-02 00:00:00,0.111,1.561,0.055,0.090,0.0,0.036,0.130,0.143,0.009,0.056,...,0.405,0.075,0.004,0.163,0.038,0.041,0.120,0.129,0.141,0.097
2010-07-02 01:00:00,0.346,0.170,0.068,0.088,0.0,0.036,0.068,0.142,0.008,0.049,...,0.413,0.056,0.000,0.203,0.062,0.048,0.106,0.110,1.413,0.924


,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User291_lag_24hrs,User292_lag_24hrs,User293_lag_24hrs,User294_lag_24hrs,User295_lag_24hrs,User296_lag_24hrs,User297_lag_24hrs,User298_lag_24hrs,User299_lag_24hrs,User300_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2013-06-30 22:00:00,0.274,0.303,0.056,0.134,0.390,1.063,0.084,0.203,0.072,0.092,...,0.332,0.131,0.221,1.592,0.444,0.162,0.226,0.558,0.800,0.35
2013-06-30 23:00:00,0.166,0.158,0.065,0.140,1.068,1.044,0.075,0.114,0.074,0.041,...,0.265,0.044,0.201,2.042,1.138,0.114,0.274,0.283,0.437,0.17


In [16]:
# Save final dataset
gross_load.to_csv('../Final_Grossload_dataset.csv')

# Final Total Load Dataset

['hour sin', 'hour cos', 'temp', 'rhum', 'wspd', 'PC1', 'lag_24hrs']

In [17]:
total_load = pd.read_csv(cwd+'/data/1process_data/2010-2013 Merged_Weather_Totalload_dataset.csv', index_col='Date')
total_load.index = pd.to_datetime(total_load.index)
total_load

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User297,User298,User299,User300,temp,dwpt,rhum,wdir,wspd,pres
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-01 00:00:00,1.200,0.179,0.065,0.078,1.859,0.036,1.794,0.146,0.113,0.818,...,0.279,0.129,0.141,2.511,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 01:00:00,1.715,0.142,2.684,0.086,1.924,0.041,1.853,0.088,0.019,0.051,...,0.106,0.110,1.413,3.316,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 02:00:00,0.865,0.119,0.683,2.034,0.092,0.046,1.715,0.097,0.397,1.125,...,0.270,0.105,1.445,0.066,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 03:00:00,1.329,0.088,0.705,0.075,0.141,0.045,0.065,0.267,0.009,0.040,...,0.902,0.063,1.457,0.084,-0.7,-1.5,94.0,360.0,7.6,1023.0
2010-07-01 04:00:00,0.098,0.057,1.791,0.081,0.888,0.045,0.078,0.188,0.073,0.047,...,0.222,0.064,1.418,0.093,-0.7,-1.5,94.0,360.0,7.6,1023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-06-30 19:00:00,0.335,0.465,0.052,0.280,2.152,1.018,0.066,0.563,0.089,0.198,...,0.233,0.366,1.071,1.405,11.8,9.9,88.0,250.0,3.6,1025.0
2013-06-30 20:00:00,0.213,0.317,0.050,0.273,1.993,0.910,0.103,0.104,0.053,0.873,...,0.539,0.318,0.992,0.625,11.8,9.9,88.0,250.0,3.6,1025.0
2013-06-30 21:00:00,0.190,0.224,0.112,0.253,0.814,1.011,0.076,0.086,0.048,0.113,...,0.298,0.272,0.840,0.695,8.8,8.7,99.0,30.0,1.8,1025.4


In [18]:
check_data(total_load)
total_load[['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres']] = total_load[['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres']].fillna(0)
check_data(total_load)

Users with negative values or NaN:
Negative counts:
Index(['temp', 'dwpt'], dtype='object')
NaN counts:
Index(['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres'], dtype='object')
Users with negative values or NaN:
Negative counts:
Index(['temp', 'dwpt'], dtype='object')
NaN counts:
Index([], dtype='object')


In [19]:
#add PCA component
total_load = pca_analysis(total_load, n_components=1)

#sine cosine scaling
total_load = sine_cosine_scaling(total_load)

#Drop unsignificant weather features
total_load.drop(columns=["dwpt", "wdir", "pres"], inplace=True)

#Introduce Lag Feature (24 hours)
total_load = lag_features(total_load)

C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[lag_col_name] = df_new[user_col].shift(lag)
C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[lag_col_name] = df_new[user_col].shift(lag)
C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [20]:
display(total_load.head(2))
display(total_load.tail(2))

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User291_lag_24hrs,User292_lag_24hrs,User293_lag_24hrs,User294_lag_24hrs,User295_lag_24hrs,User296_lag_24hrs,User297_lag_24hrs,User298_lag_24hrs,User299_lag_24hrs,User300_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-02 00:00:00,1.199,1.561,0.055,0.090,2.154,0.036,1.837,0.143,0.009,1.078,...,0.405,0.075,1.792,2.407,0.038,0.041,0.279,0.129,0.141,2.511
2010-07-02 01:00:00,1.584,0.170,2.085,0.088,0.518,0.036,1.808,0.142,0.008,0.049,...,0.413,0.056,0.938,1.387,0.062,0.048,0.106,0.110,1.413,3.316


,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User291_lag_24hrs,User292_lag_24hrs,User293_lag_24hrs,User294_lag_24hrs,User295_lag_24hrs,User296_lag_24hrs,User297_lag_24hrs,User298_lag_24hrs,User299_lag_24hrs,User300_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2013-06-30 22:00:00,0.274,0.303,0.056,0.134,0.390,1.063,0.084,0.203,0.072,0.092,...,0.332,0.131,0.221,1.592,0.444,0.162,0.226,0.558,0.800,0.35
2013-06-30 23:00:00,0.166,0.158,0.065,0.140,1.068,1.044,0.075,0.827,0.074,0.041,...,0.265,0.044,0.201,2.042,1.138,0.114,2.644,0.283,0.437,0.17


In [21]:
# Save final dataset
total_load.to_csv('../Final_Totalload_dataset.csv')

# Final Prosumption Dataset

['hour sin', 'hour cos', 'temp', 'rhum', 'wspd', 'PC1', 'lag_24hrs']

In [22]:
prosumption = pd.read_csv(cwd+'/data/1process_data/2010-2013 Merged_Weather_Prosumption_dataset.csv', index_col='Date')
prosumption.index = pd.to_datetime(prosumption.index)
prosumption

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User297,User298,User299,User300,temp,dwpt,rhum,wdir,wspd,pres
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-01 00:00:00,1.200,0.179,0.065,0.078,1.859,0.036,1.794,0.146,0.113,0.818,...,0.279,0.129,0.141,2.511,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 01:00:00,1.715,0.142,2.684,0.086,1.924,0.041,1.853,0.088,0.019,0.051,...,0.106,0.110,1.413,3.316,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 02:00:00,0.865,0.119,0.683,2.034,0.092,0.046,1.715,0.097,0.397,1.125,...,0.270,0.105,1.445,0.066,0.7,-0.6,91.0,160.0,5.4,1022.0
2010-07-01 03:00:00,1.329,0.088,0.705,0.075,0.141,0.045,0.065,0.267,0.009,0.040,...,0.902,0.063,1.457,0.084,-0.7,-1.5,94.0,360.0,7.6,1023.0
2010-07-01 04:00:00,0.098,0.057,1.791,0.081,0.882,0.045,0.078,0.188,0.073,0.047,...,0.222,0.064,1.418,0.093,-0.7,-1.5,94.0,360.0,7.6,1023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-06-30 19:00:00,0.335,0.465,0.046,0.280,2.152,1.018,0.066,0.563,0.089,0.198,...,0.233,0.366,1.071,1.405,11.8,9.9,88.0,250.0,3.6,1025.0
2013-06-30 20:00:00,0.213,0.317,0.050,0.273,1.993,0.910,0.103,0.104,0.053,0.873,...,0.539,0.318,0.992,0.625,11.8,9.9,88.0,250.0,3.6,1025.0
2013-06-30 21:00:00,0.190,0.224,0.112,0.253,0.814,1.011,0.076,0.086,0.048,0.113,...,0.298,0.272,0.840,0.695,8.8,8.7,99.0,30.0,1.8,1025.4


In [23]:
check_data(prosumption)
prosumption[['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres']] = prosumption[['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres']].fillna(0)
check_data(prosumption)

Users with negative values or NaN:
Negative counts:
Index(['User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7', 'User8',
       'User9', 'User10',
       ...
       'User293', 'User294', 'User295', 'User296', 'User297', 'User298',
       'User299', 'User300', 'temp', 'dwpt'],
      dtype='object', length=302)
NaN counts:
Index(['User2', 'temp', 'dwpt', 'rhum', 'wdir', 'pres'], dtype='object')
Users with negative values or NaN:
Negative counts:
Index(['User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7', 'User8',
       'User9', 'User10',
       ...
       'User293', 'User294', 'User295', 'User296', 'User297', 'User298',
       'User299', 'User300', 'temp', 'dwpt'],
      dtype='object', length=302)
NaN counts:
Index([], dtype='object')


In [24]:
#add PCA component
prosumption = pca_analysis(prosumption, n_components=1)

#sine cosine scaling
prosumption = sine_cosine_scaling(prosumption)

#Drop unsignificant weather features
prosumption.drop(columns=["dwpt", "wdir", "pres"], inplace=True)

#Introduce Lag Feature (24 hours)
prosumption = lag_features(prosumption)

C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[lag_col_name] = df_new[user_col].shift(lag)
C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[lag_col_name] = df_new[user_col].shift(lag)
C:\Users\rs1044\AppData\Local\Temp\ipykernel_27892\1457363175.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [25]:
display(prosumption.head(2))
display(prosumption.tail(2))

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User291_lag_24hrs,User292_lag_24hrs,User293_lag_24hrs,User294_lag_24hrs,User295_lag_24hrs,User296_lag_24hrs,User297_lag_24hrs,User298_lag_24hrs,User299_lag_24hrs,User300_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-02 00:00:00,1.199,1.561,0.055,0.090,2.154,0.036,1.837,0.143,0.009,1.078,...,0.405,0.075,1.792,2.407,0.038,0.041,0.279,0.129,0.141,2.511
2010-07-02 01:00:00,1.584,0.170,2.085,0.088,0.518,0.036,1.808,0.142,0.008,0.049,...,0.413,0.056,0.938,1.387,0.062,0.048,0.106,0.110,1.413,3.316


,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User291_lag_24hrs,User292_lag_24hrs,User293_lag_24hrs,User294_lag_24hrs,User295_lag_24hrs,User296_lag_24hrs,User297_lag_24hrs,User298_lag_24hrs,User299_lag_24hrs,User300_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2013-06-30 22:00:00,0.274,0.303,0.056,0.134,0.390,1.063,0.084,0.203,0.072,0.092,...,0.332,0.131,0.221,1.592,0.444,0.162,0.226,0.558,0.800,0.35
2013-06-30 23:00:00,0.166,0.158,0.065,0.140,1.068,1.044,0.075,0.827,0.074,0.041,...,0.265,0.044,0.201,2.042,1.138,0.114,2.644,0.283,0.437,0.17


In [26]:
# Save final dataset
prosumption.to_csv('../Final_Prosumption_dataset.csv')